# 🚀 ISL-123 Training - Complete & Working

**Everything in one place - Auto-generates labels, filters correctly, trains with augmentation**

## What this notebook does:
1. ✅ Auto-generates `file_to_label.json` from INCLUDE dataset
2. ✅ **Filters to ONLY the 123 target classes** (not 158!)
3. ✅ Removes classes with <2 samples (stratification requirement)
4. ✅ Creates stratified 80/20 train/val split
5. ✅ Trains with 7 augmentation techniques
6. ✅ Targets **73-78% validation accuracy**

## Required Inputs:
- `INCLUDE` dataset (source videos)
- `isl-123-cache` (pre-extracted landmarks)

## Expected Runtime:
- Label generation: ~30 seconds
- Training: ~6-8 minutes
- **Total: ~7-9 minutes**

---

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 1: Setup
# ═══════════════════════════════════════════════════════════
import numpy as np
import json
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random
from sklearn.model_selection import train_test_split

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)
print("\n✅ Setup complete")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 2: Configuration
# ═══════════════════════════════════════════════════════════
CONFIG = {
    # Data
    'seq_len': 60,
    'input_dim': 408,
    'num_classes': 123,
    'train_split': 0.80,
    
    # Model
    'hidden_dim': 512,
    'num_heads': 8,
    'num_layers': 4,
    'dropout': 0.3,
    
    # Training
    'batch_size': 32,
    'epochs': 100,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'label_smoothing': 0.1,
    'gradient_clip': 1.0,
    'patience': 15,
    'min_delta': 0.001,
    
    # Augmentation (moderate strength)
    'aug_time_warp_prob': 0.3,
    'aug_time_warp_range': (0.9, 1.1),
    'aug_noise_prob': 0.4,
    'aug_noise_std': 0.015,
    'aug_rotation_prob': 0.3,
    'aug_rotation_range': (-10, 10),
    'aug_scaling_prob': 0.3,
    'aug_scaling_range': (0.95, 1.05),
    'aug_masking_prob': 0.2,
    'aug_masking_ratio': 0.05,
    'aug_temporal_shift_prob': 0.2,
    'aug_mixup_prob': 0.1,
    'aug_mixup_alpha': 0.1,
    
    # Paths
    'include_dir': '/kaggle/input/include',
    'cache_dir': '/kaggle/input/isl-123-cache/isl_cache_123',
    'mapping_file': '/kaggle/input/isl-123-cache/label_mapping_123.json',
    'save_dir': '/kaggle/working'
}

print("✅ Config loaded")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 3: Load Target Classes (123 classes only)
# ═══════════════════════════════════════════════════════════
print("="*60)
print("📋 LOADING TARGET CLASSES")
print("="*60)

with open(CONFIG['mapping_file']) as f:
    mapping = json.load(f)
    label_to_id = mapping['label_to_id']
    id_to_label = {int(k): v for k, v in mapping['id_to_label'].items()}

# Create set of target classes (lowercase)
target_classes = set(cls.lower() for cls in label_to_id.keys())

print(f"\n✅ Target: {len(target_classes)} classes")
print(f"\nFirst 10:")
for i, cls in enumerate(sorted(target_classes)[:10]):
    print(f"   {i+1}. {cls}")

print(f"\n✅ Will filter INCLUDE to ONLY these {len(target_classes)} classes")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 4: Auto-Generate file_to_label.json (FILTERED!)
# ═══════════════════════════════════════════════════════════
print("\n" + "="*60)
print("🔧 AUTO-GENERATING file_to_label.json")
print("="*60)

INCLUDE_DIR = Path(CONFIG['include_dir'])
CACHE_DIR = Path(CONFIG['cache_dir'])

# Get cache files
cache_files = {f.stem: f.name for f in CACHE_DIR.glob('*.npy')}
print(f"\n📦 Cache files: {len(cache_files)}")

# Scan INCLUDE (FILTERED to target classes only!)
print(f"\n🔍 Scanning INCLUDE (filtering to 123 target classes)...")
video_to_class = {}

for root, dirs, files in os.walk(INCLUDE_DIR):
    for file in files:
        if file.lower().endswith(('.mov', '.mp4')):
            relative = Path(root).relative_to(INCLUDE_DIR)
            parts = relative.parts
            
            if len(parts) >= 3:
                class_folder = parts[2]
                
                if '. ' in class_folder:
                    class_name = class_folder.split('. ', 1)[1].strip().lower()
                else:
                    class_name = class_folder.strip().lower()
                
                # ✅ KEY FIX: Only include if in target 123 classes!
                if class_name in target_classes:
                    video_stem = Path(file).stem
                    video_to_class[video_stem] = class_name

print(f"✅ Found {len(video_to_class)} videos in target 123 classes")

# Match cache → classes
print(f"\n🔗 Matching cache files...")
file_to_label_all = {}

for cache_stem, cache_name in tqdm(cache_files.items(), desc="Matching"):
    if cache_stem in video_to_class:
        file_to_label_all[cache_name] = video_to_class[cache_stem]
    else:
        # Partial match
        for video_stem, class_name in video_to_class.items():
            if video_stem in cache_stem or cache_stem in video_stem:
                file_to_label_all[cache_name] = class_name
                break

print(f"\n📊 Matched: {len(file_to_label_all)}/{len(cache_files)}")

# ✅ Filter out classes with <2 samples (stratification requires ≥2)
class_counts = Counter(file_to_label_all.values())
classes_to_keep = {cls for cls, count in class_counts.items() if count >= 2}

file_to_label = {
    fname: label 
    for fname, label in file_to_label_all.items() 
    if label in classes_to_keep
}

removed = len(file_to_label_all) - len(file_to_label)
if removed > 0:
    print(f"\n🧹 Filtered out {removed} samples (classes with <2 samples)")

# Final stats
class_counts = Counter(file_to_label.values())

print(f"\n📈 Final Results:")
print(f"   Samples: {len(file_to_label)}")
print(f"   Classes: {len(class_counts)}/{len(target_classes)}")
print(f"   Min/class: {min(class_counts.values())}")
print(f"   Max/class: {max(class_counts.values())}")
print(f"   Avg/class: {len(file_to_label)/len(class_counts):.1f}")

# Validation
if min(class_counts.values()) < 2:
    raise ValueError("❌ Some classes have <2 samples - can't stratify!")

if len(class_counts) < 100:
    print(f"\n⚠️  Only {len(class_counts)} classes (expected ~123)")

# Save
with open(Path(CONFIG['save_dir']) / 'file_to_label.json', 'w') as f:
    json.dump(file_to_label, f, indent=2, sort_keys=True)

print(f"\n💾 Saved: file_to_label.json")
print(f"✅ Filtering complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 5: Stratified Split
# ═══════════════════════════════════════════════════════════
print("\n" + "="*60)
print("✂️  STRATIFIED SPLIT")
print("="*60)

filenames = list(file_to_label.keys())
labels = [file_to_label[f] for f in filenames]

train_files, val_files, train_labels, val_labels = train_test_split(
    filenames, labels,
    test_size=1 - CONFIG['train_split'],
    stratify=labels,
    random_state=42
)

train_file_to_label = {f: l for f, l in zip(train_files, train_labels)}
val_file_to_label = {f: l for f, l in zip(val_files, val_labels)}

# Verify
overlap = set(train_files) & set(val_files)
print(f"\n🔍 Verification:")
print(f"   Train: {len(train_files)}")
print(f"   Val: {len(val_files)}")
print(f"   Overlap: {len(overlap)} {'✅' if len(overlap) == 0 else '❌'}")

if overlap:
    raise ValueError("Data leakage!")

print(f"\n✅ Split OK")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 6: Augmentation
# ═══════════════════════════════════════════════════════════
class Augmentation:
    def __init__(self, config):
        self.c = config
    
    def time_warp(self, x):
        if np.random.rand() > self.c['aug_time_warp_prob']: return x
        scale = np.random.uniform(*self.c['aug_time_warp_range'])
        new_len = int(len(x) * scale)
        new_len = np.clip(new_len, len(x)//2, len(x)*2)
        idx = np.linspace(0, len(x)-1, new_len).astype(int)
        return x[np.clip(idx, 0, len(x)-1)]
    
    def add_noise(self, x):
        if np.random.rand() > self.c['aug_noise_prob']: return x
        return x + np.random.normal(0, self.c['aug_noise_std'], x.shape).astype(np.float32)
    
    def rotate(self, x):
        if np.random.rand() > self.c['aug_rotation_prob']: return x
        angle = np.radians(np.random.uniform(*self.c['aug_rotation_range']))
        R = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        rotated = x.copy()
        for i in range(0, x.shape[1], 3):
            if i+1 < x.shape[1]:
                rotated[:, i:i+2] = x[:, i:i+2] @ R.T
        return rotated.astype(np.float32)
    
    def scale(self, x):
        if np.random.rand() > self.c['aug_scaling_prob']: return x
        s = np.random.uniform(*self.c['aug_scaling_range'])
        return (x * s).astype(np.float32)
    
    def mask(self, x):
        if np.random.rand() > self.c['aug_masking_prob']: return x
        n = int(len(x) * self.c['aug_masking_ratio'])
        if n > 0: x[np.random.choice(len(x), n, replace=False)] = 0
        return x
    
    def shift(self, x):
        if np.random.rand() > self.c['aug_temporal_shift_prob']: return x
        s = np.random.randint(-len(x)//10, len(x)//10+1)
        if s > 0: x = np.vstack([np.repeat(x[:1], s, 0), x[:-s]])
        elif s < 0: x = np.vstack([x[-s:], np.repeat(x[-1:], -s, 0)])
        return x
    
    def apply(self, x):
        for fn in [self.time_warp, self.add_noise, self.rotate, self.scale, self.mask, self.shift]:
            x = fn(x)
        return x

print("✅ Augmentation ready")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 7: Dataset
# ═══════════════════════════════════════════════════════════
class ISLDataset(Dataset):
    def __init__(self, cache_dir, file_to_label, label_to_id, seq_len=60, augment=False, config=None):
        self.cache_dir = Path(cache_dir)
        self.seq_len = seq_len
        self.label_to_id = label_to_id
        self.aug = Augmentation(config) if augment else None
        self.samples = [(self.cache_dir/f, l) for f, l in file_to_label.items() if (self.cache_dir/f).exists()]
        print(f"   {len(self.samples)} samples, aug={augment}")
    
    def __len__(self): 
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        x = np.load(path).astype(np.float32)
        
        if self.aug: x = self.aug.apply(x)
        
        # Pad/trim
        if len(x) >= self.seq_len:
            idx = np.linspace(0, len(x)-1, self.seq_len, dtype=int)
            x = x[idx]
        else:
            x = np.vstack([x, np.repeat(x[-1:], self.seq_len-len(x), 0)])
        
        # Normalize
        x = (x - x.mean(0, keepdims=True)) / (x.std(0, keepdims=True) + 1e-8)
        
        return torch.from_numpy(x), self.label_to_id[label]

# Create datasets
print("\n📦 Creating datasets...")
train_ds = ISLDataset(CONFIG['cache_dir'], train_file_to_label, label_to_id, CONFIG['seq_len'], True, CONFIG)
val_ds = ISLDataset(CONFIG['cache_dir'], val_file_to_label, label_to_id, CONFIG['seq_len'], False, CONFIG)

train_loader = DataLoader(train_ds, CONFIG['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)

print(f"\n✅ Train: {len(train_ds)} samples, {len(train_loader)} batches")
print(f"✅ Val: {len(val_ds)} samples, {len(val_loader)} batches")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 8: Model
# ═══════════════════════════════════════════════════════════
class ISLModel(nn.Module):
    def __init__(self, input_dim=408, hidden=512, classes=123, heads=8, layers=4, drop=0.3):
        super().__init__()
        self.embed = nn.Sequential(nn.Linear(input_dim, hidden), nn.LayerNorm(hidden), nn.GELU(), nn.Dropout(drop))
        enc = nn.TransformerEncoderLayer(hidden, heads, hidden*4, drop, 'gelu', batch_first=True, norm_first=True)
        self.transformer = nn.TransformerEncoder(enc, layers)
        self.classifier = nn.Sequential(nn.Linear(hidden, hidden//2), nn.GELU(), nn.Dropout(drop), nn.Linear(hidden//2, classes))
    
    def forward(self, x):
        return self.classifier(self.transformer(self.embed(x)).mean(1))

model = ISLModel(
    CONFIG['input_dim'], CONFIG['hidden_dim'], CONFIG['num_classes'],
    CONFIG['num_heads'], CONFIG['num_layers'], CONFIG['dropout']
).to(device)

print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print("✅ Model ready")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 9: Training Functions
# ═══════════════════════════════════════════════════════════
criterion = nn.CrossEntropyLoss(label_smoothing=CONFIG['label_smoothing'])
optimizer = torch.optim.AdamW(model.parameters(), CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

def train_epoch(model, loader):
    model.train()
    total_loss = correct = total = 0
    for x, y in tqdm(loader, desc='Train'):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['gradient_clip'])
        optimizer.step()
        total_loss += loss.item()
        correct += out.argmax(1).eq(y).sum().item()
        total += len(y)
    return total_loss/len(loader), 100*correct/total

def validate(model, loader):
    model.eval()
    total_loss = correct = total = 0
    with torch.no_grad():
        for x, y in tqdm(loader, desc='Val'):
            x, y = x.to(device), y.to(device)
            out = model(x)
            total_loss += criterion(out, y).item()
            correct += out.argmax(1).eq(y).sum().item()
            total += len(y)
    return total_loss/len(loader), 100*correct/total

print("✅ Training functions ready")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 10: Main Training Loop
# ═══════════════════════════════════════════════════════════
print("\n" + "="*60)
print("🚀 TRAINING")
print("="*60)
print("🎯 Target: 73-78% val accuracy\n")

best_acc = 0
best_epoch = 0
patience = 0
history = {'train_loss':[], 'train_acc':[], 'val_loss':[], 'val_acc':[], 'lr':[]}
start = time.time()

for epoch in range(CONFIG['epochs']):
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
    
    t_loss, t_acc = train_epoch(model, train_loader)
    v_loss, v_acc = validate(model, val_loader)
    scheduler.step()
    
    history['train_loss'].append(t_loss)
    history['train_acc'].append(t_acc)
    history['val_loss'].append(v_loss)
    history['val_acc'].append(v_acc)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    print(f"Train: {t_loss:.4f} / {t_acc:.2f}% | Val: {v_loss:.4f} / {v_acc:.2f}%")
    
    if v_acc > best_acc + CONFIG['min_delta']:
        best_acc = v_acc
        best_epoch = epoch + 1
        patience = 0
        torch.save({
            'epoch': epoch+1, 'model': model.state_dict(), 'acc': best_acc,
            'config': CONFIG, 'history': history
        }, f"{CONFIG['save_dir']}/best_isl_123.pth")
        print(f"✅ NEW BEST: {v_acc:.2f}%")
        if v_acc >= 75: print("🎯 TARGET!")
    else:
        patience += 1
        print(f"({patience}/{CONFIG['patience']})")
    
    if patience >= CONFIG['patience']:
        print(f"\nEarly stop at epoch {epoch+1}")
        break

elapsed = time.time() - start
print("\n" + "="*60)
print("✅ COMPLETE")
print("="*60)
print(f"Best: {best_acc:.2f}% (epoch {best_epoch})")
print(f"Time: {elapsed/60:.2f} min")

with open(f"{CONFIG['save_dir']}/history.json", 'w') as f:
    json.dump(history, f, indent=2)

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 11: Plot Results
# ═══════════════════════════════════════════════════════════
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], 'o-', label='Train', alpha=0.8)
ax1.plot(history['val_loss'], 's-', label='Val', alpha=0.8)
ax1.axvline(best_epoch-1, color='r', linestyle='--', alpha=0.5)
ax1.set(xlabel='Epoch', ylabel='Loss', title='Loss')
ax1.legend(); ax1.grid(alpha=0.3)

ax2.plot(history['train_acc'], 'o-', label='Train', alpha=0.8)
ax2.plot(history['val_acc'], 's-', label='Val', alpha=0.8)
ax2.axvline(best_epoch-1, color='r', linestyle='--', alpha=0.5)
ax2.axhline(best_acc, color='g', linestyle=':', alpha=0.5)
ax2.axhline(75, color='purple', linestyle='--', alpha=0.5, label='Target')
ax2.set(xlabel='Epoch', ylabel='Accuracy (%)', title=f'Accuracy (Best: {best_acc:.2f}%)')
ax2.legend(); ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CONFIG['save_dir']}/results.png", dpi=150)
plt.show()

print("✅ Plotted")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 12: Summary
# ═══════════════════════════════════════════════════════════
print("="*60)
print("📋 SUMMARY")
print("="*60)
print(f"\n✅ Best Val Acc: {best_acc:.2f}% (Epoch {best_epoch})")
print(f"✅ Time: {elapsed/60:.2f} minutes")
print(f"✅ Classes: {len(class_counts)}")
print(f"✅ Samples: {len(file_to_label)}")

print(f"\n💾 Saved:")
print(f"   - file_to_label.json")
print(f"   - best_isl_123.pth")
print(f"   - history.json")
print(f"   - results.png")

if best_acc >= 75:
    print(f"\n🎉 SUCCESS! {best_acc:.2f}% >= 75%")
elif best_acc >= 73:
    print(f"\n👍 VERY CLOSE! {best_acc:.2f}%")
elif best_acc > 70:
    print(f"\n📈 IMPROVED! {best_acc:.2f}% (baseline 70%)")

print("\n" + "="*60)
print("✅ DONE")
print("="*60)